# Step 1: Instruction Fine-Tuning GPT-2

In [ ]:
!pip install datasets #허깅페이스
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, pipeline, \
                         AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from datasets import Dataset, load_dataset
import torch
import json

In [ ]:
seed=42

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
device = torch.device("cuda")

device

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
data_path = '/content/drive/MyDrive/fine tuning/chip2.csv'
training_df = pd.read_csv(data_path)                #####csv를 데이터프레임으로 변환


![Screenshot%202023-08-02%20at%206.48.20%20AM.png](attachment:Screenshot%202023-08-02%20at%206.48.20%20AM.png)

Find it [here](https://laion.ai/blog/oig-dataset/)

In [ ]:
training_df.head()

In [ ]:
print(training_df.iloc[798]['prompt'])            ##### iloc는 798은 데이터프레임의 799번째 행
print("========")
print(training_df.iloc[798]['response'])
print("========")


In [ ]:
training_df.head()

In [ ]:
training_df.shape

In [ ]:
training_df['response'] = training_df['response'].map(lambda x: x.strip())   ##### map은 각요소에 함수적용 , 익명함수 람다, 각요소x에대해 strip()공백제거
training_df['prompt'] = training_df['prompt'].map(lambda x: x.strip())


In [ ]:
training_df = training_df.drop_duplicates(subset=['prompt'])                 ##### prompt 열을 기준으로 중복을 제거

In [ ]:
training_df.shape

In [ ]:
training_df.head(2)

In [ ]:
training_df['source'].value_counts().plot(kind='bar')                      ##### 종류별분류

In [ ]:
training_df.shape

In [ ]:
training_df['prompt'].str.len().plot(kind='hist', title='Histogram of prompt Length')

In [ ]:
training_df['response'].str.len().plot(kind='hist', title='Histogram of response Length')


  
# 모델학습 시작


In [ ]:
QUERY_KEY = "Question:"
RESPONSE_KEY = "Response:"
PAD_KEY = "<PAD>"


In [ ]:

MODEL = 'gpt2'

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# 사용자토큰정의
# 일반적인 특수 토큰(e.g., <pad>, <cls>) 외에 추가로 사용자 정의 특수 토큰을 포함
# 질문,답변을 구분 / 예:"What is AI?"를 "<QUERY> What is AI?"로 변환. / 입력 시퀀스의 길이를 맞추기 위해 패딩 토큰을 사용
tokenizer.add_special_tokens({"additional_special_tokens": [QUERY_KEY, RESPONSE_KEY, PAD_KEY]})
tokenizer.pad_token = PAD_KEY


model = AutoModelForCausalLM.from_pretrained(MODEL)
model.config.pad_token = PAD_KEY
model.resize_token_embeddings(len(tokenizer))


In [ ]:
chip2_dataset = Dataset.from_pandas(training_df)                ##### 허깅페이스 Dataset라이브러리로 pandas 데이터프레임을 처리

# 사용자토큰 적용함수
def preprocess(example):
    return tokenizer(QUERY_KEY+ ' ' + example['prompt'] + '\n'+RESPONSE_KEY+' '+example['response']+tokenizer.eos_token)

chip2_dataset = chip2_dataset.map(
    preprocess, batched=False, batch_size=1024, remove_columns=chip2_dataset.features.keys()
)

# 길이조정함수
def filter_function(example):
    return len(example['input_ids']) <= 1024

chip2_dataset = chip2_dataset.filter(filter_function)

chip2_dataset = chip2_dataset.train_test_split(test_size=0.2, seed=seed)

In [ ]:
chip2_dataset

In [ ]:
print(tokenizer.decode(chip2_dataset['test']['input_ids'][13]))
print('-----')
print(tokenizer.decode(chip2_dataset['test']['input_ids'][14]))
print('-----')
print(tokenizer.decode(chip2_dataset['test']['input_ids'][63]))

In [ ]:
len(chip2_dataset['test']['input_ids'][0])

In [ ]:
f'{model.num_parameters():,}'

In [ ]:
import wandb                  #### 모델학습 추적, 시각화
# Set up Weights and Biases integration
wandb.init(project="FFT") #b93a105e31d833b072c1f15f57a095f50c62e0d3


In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
print(torch.cuda.get_device_name(0))
if device.type == 'cuda':
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
    return_tensors="pt", pad_to_multiple_of=8
)

In [ ]:
from transformers import TrainingArguments, Trainer

##### 실험설정 다르게 2번돌림
training_args = TrainingArguments(
    output_dir="./fineframe_supervised_instruction",  # The output directory
    overwrite_output_dir=True,  # overwrite the content of the output directory
    num_train_epochs=1,  # number of training epochs
    per_device_train_batch_size=2,  # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    gradient_accumulation_steps=16,  # steps for gradient accumulation
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    report_to="all",
    seed=seed,
    fp16=True,  # enable mixed precision training for my GPU
)

# GPT2모델을 chip2데이터셋으로 훈련시킨다 (fine tuning)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=chip2_dataset['train'],
    eval_dataset=chip2_dataset['test'],
    data_collator=data_collator
)

trainer.evaluate()


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./fineframe",  # The output directory
    overwrite_output_dir=True,  # overwrite the content of the output directory
    num_train_epochs=1,  # number of training epochs
    per_device_train_batch_size=2,  # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    gradient_accumulation_steps=16,  # steps for gradient accumulation
    logging_steps=50,
    learning_rate=2e-5,
    warmup_steps=100,
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    report_to="all",
    seed=seed,
    fp16=True,  # enable mixed precision training for GPU
)

##### transformers라이브러리 Trainer에 모델, args, 데이터셋 넣고 train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=chip2_dataset['train'],
    eval_dataset=chip2_dataset['test'],
    data_collator=data_collator
)

trainer.evaluate()


In [ ]:
number_of_examples = len(chip2_dataset['train'])
per_device_train_batch_size = training_args.per_device_train_batch_size
gradient_accumulation_steps = training_args.gradient_accumulation_steps
num_train_epochs = training_args.num_train_epochs

effective_batch_size = per_device_train_batch_size * gradient_accumulation_steps
number_of_training_steps = (number_of_examples / effective_batch_size) * num_train_epochs

number_of_training_steps = int(number_of_training_steps / training_args.n_gpu)

print("Number of training steps:", number_of_training_steps)


In [ ]:
training_args.warmup_steps = int(.1 * number_of_training_steps)

In [ ]:
trainer.train()

In [ ]:
# 모델 저장
model_save_path = "/content/drive/My Drive/fine tuning/trained_model"
tokenizer_save_path = "/content/drive/My Drive/fine tuning/trained_model/tokenizer"

trainer.save_model(model_save_path)

In [ ]:
tokenizer.save_pretrained(tokenizer_save_path)


**모델을 구글드라이브에 저장된것을 불러옴**




In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 저장된 경로
model_load_path = "/content/drive/My Drive/fine tuning/trained_model"
tokenizer_load_path = "/content/drive/My Drive/fine tuning/trained_model/tokenizer"

# 모델 및 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(model_load_path)    # 처음엔 model = AutoModelForCausalLM.from_pretrained("GPT2")
# 모델사용 목적이다름, CausalLM 모델은 언어 생성에 필요한 자동 회귀 방식으로 학습됩니다.
# Sequence Classification 모델은 분류 작업을 위한 엔드 투 엔드 분류 구조로 학습됩니다. AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained(tokenizer_load_path)

### Loss drops dramatically which is expected when we introduce new tokens
